# Make labels

The labels from the highest quality assignment for each site were rasterized into 3 classes: non-field (0), field interior (1), and field edge, which provides the basis for training a boundary aware semantic segmentation model that can be used to more effectively separate individual fields. 

The resulting chips provide a single label for each image chip. Additional assignments for Class 1 and 4 sites are also available, and can be converted to chips for different purposes (e.g. to assess the impact of label noise on model predictions; Elmes et al, 2020). 

This notebook may also be adapted to create different types of labels, such as binary labels (field/no-field) or separate labels representing, for example, field edge, field interior, non-field, and distance to nearest field boundary (cite).

In [1]:
import os
import re
import warnings
from pathlib import Path
import rioxarray as rxr
import xarray as xr
from rasterio.enums import Resampling
from rasterio import features
import geopandas as gpd
from shapely.geometry import Polygon, box, mapping
import pandas as pd
import numpy as np
from datetime import datetime as dt
import multiprocessing as mp
import leafmap.leafmap as leafmap

## Set-up

In [2]:
root_dir = os.environ["HOME"]
proj_dir = Path(root_dir) / "Dropbox/projects/lacunalabels"
data_dir = Path(root_dir) / "labels"
chip_dir = Path(data_dir) / "lacuna/images"
label_dir = Path(data_dir) / "lacuna/labels"
final_dir = Path(proj_dir) / "data/processed"
if not os.path.exists(label_dir):
    os.makedirs(label_dir, exist_ok=True)
log_path = Path(root_dir) / "logs/label-maker2.log"

### Functions

In [3]:
def filter_catalog(catalog, groups, metric, keep):
    """
    Function to filter the full catalog by class and quality metric

    Args:
    catalog: DataFrame
        The full catalog, read in
    groups: List
        A list of key-value pairs providing with possible keys of "whole" and 
        "best", with the values providing one or more of the label classes. 
        Classes corresponding to "whole" will have all assignments in the class
        selected. "Best" will result in the best assignment 
        corresponding to the provided metric selected/
    metric: str
        One of the quality metrics in the catalog, e.g. Rscore, Qscore. Must be 
        provided if a key in groups is not "whole"
    keep: list
        Names of columns in the full catalog that should be kept in the 
        filtered catalog

    Returns:
        A DataFrame containing the filtered assignments, possibly with 
        duplicates. If so, you may wish to remove them by following up with a 
        `drop_duplicates()`
    """
    out_catalog = []
    for g in groups:
        cls = list(g.values())[0]
        cat = catalog.query("Class in @cls")
        if list(g.keys())[0] == "whole":
            print(f"Extracting all of Class {' and '.join(cls)}")
            out_catalog.append(cat)
        elif list(g.keys())[0] == "best": 
            print(f"Extracting best of Class {' and '.join(cls)}")
            out_catalog.append(
                cat.groupby("name")
                .apply(lambda x: x.loc[[x[metric].idxmax()]] 
                       if not x[metric].isna().all() else x, 
                       include_groups=False)
                .reset_index(level=["name"])
            )
        else: 
            print("Use either 'whole' or 'best' as group keys")
            break 

    out_catalog = pd.concat(out_catalog, axis=0)[keep].reset_index(drop=True)
    return out_catalog

def threeclass_label(assignment, label_dir, fields, log=None, 
                     overwrite=True):
    """
    Create a three class label (0: non-field, 1: field interior, 
    2: field boundary) with the same dimensions as the corresponding 
    image chip. 

    Args: 
    assignment: pandas.Series
        A series representing one assignment from the label catalog
    fields: geopandas.GeoDataFrame
        The fields polygons, read in from the provided geoparquet file
    label_dir: str or Path
        Directory to write rasterized labels to
    log: str or Path
        Name and path of log file
    overwrite: bool
        Overwrite label if it exists on disk or not (default = True)
    """

    lbl_name = f"{re.sub(".tif", "", assignment.chip)}-"\
        f"{assignment.assignment_id}.tif"
    dst_path = Path(label_dir) / lbl_name

    if not overwrite and os.path.exists(dst_path):
       msg = f"{os.path.basename(dst_path)} exists, skipping"
       print(msg, file=log, flush=True)

    else: 
        chip = rxr.open_rasterio(Path(chip_dir) / assignment.chip)
        
        transform = chip.rio.transform()
        _, r, c = chip.shape
        res = np.mean([abs(transform[0]), abs(transform[4])])
        
        grid = gpd.GeoDataFrame(geometry=[box(*chip.rio.bounds())], 
                                crs=chip.rio.crs)
        
        out_arr = np.zeros((r, c)).astype('int16')
        if assignment.nflds > 0:
            
            shp = fields.query("assignment_id==@row.assignment_id").copy()
    
            shp["category"] = 1
            shp['buffer_in'] = shp.geometry.buffer(-res)
            shp['buffer_out'] = shp.geometry.buffer(res)
            shp = gpd.overlay(grid, shp, how='intersection')
            out_arr = np.zeros((r, c)).astype('uint8')
        
            shapes = ((geom, value) 
                      for geom, value in zip(shp['geometry'], shp['category']))
            burned = features.rasterize(shapes=shapes, fill=0, 
                                        out=out_arr.copy(), 
                                        transform=transform)
            
            try:
                shapes_shrink = (
                    (geom, value) 
                    for geom, value in zip(shp['buffer_in'], shp['category'])
                )
                shrunk = features.rasterize(
                    shapes=shapes_shrink, fill=0, out=out_arr.copy(), 
                    transform=transform
                )
                shapes_explode = (
                    (geom, value) 
                    for geom, value in zip(shp['buffer_out'], shp['category'])
                )
                exploded = features.rasterize(
                    shapes=shapes_explode, fill=0, out=out_arr.copy(), 
                    transform=transform
                )
            except:
                shp['buffer'] = shp.geometry.buffer(-res)
                shapes_shrink = (
                    (geom, value) 
                    for geom, value in zip(shp['buffer'], shp['category'])
                )
                shrunk = features.rasterize(
                    shapes=shapes_shrink, fill=0, out=out_arr.copy(), 
                    transform=transform
                )
            
            lbl = (
                burned * 2 - shrunk + \
                np.where((exploded*2-burned)==1, 0, exploded*2-burned)
                .astype(np.uint8)
            )
        else: 
            lbl = out_arr
    
        lbl_raster = xr.DataArray(
            lbl,
            dims=["y", "x"],
            coords={"y": chip.y, "x": chip.x},
            attrs={"transform": transform, "crs": chip.rio.crs}
        )

        # check dimensions
        try:
            assert chip.rio.bounds() == lbl_raster.rio.bounds()
        except AssertionError as err:
            msg = f"{dt.now()}: {os.path.basename(dst_path)} has incorrect bounds"
            print(msg, file=log, flush=True)
            raise err
        try:    
            assert chip.shape[1:3] == lbl_raster.shape
        except AssertionError as err:
            msg = f"{dt.now()}: {os.path.basename(dst_path)} incorrect output shape"
            print(msg, file=log, flush=True)
            raise err

        # write to disk
        lbl_raster.rio.to_raster(dst_path)
        msg = f"{dt.now()}: created {os.path.basename(dst_path)}"
        print(msg, file=log, flush=True)

    assignment_out = assignment.copy()
    assignment_out["label"] = lbl_name

    return pd.DataFrame([assignment_out.to_list()], 
                        columns=assignment_out.index)

def view_random_label(label_catalog, label_dir, chip_dir, bands=[1,2,3], 
                      seed=None): 
    """
    A leafmap-based viewer that enables comparison of a randomly selected
    label against of the image chip

    Args: 
    label_catalog: pandas.DataFrame
        The processed label catalog
    label_dir: str
        The path to the label directory (not strings only, not Path)
    chip_dir: str
        The path to the image chip directory (not strings only, not Path)
    bands: list
        Specify band combination for the image chip. Defaults to [1,2,3] for 
        true color
    seed: int
        Defaults to None, but the same chip can be selected again if an integer
        is provided

    Returns: 
        A leafmap viewer
    """
    random_label = label_catalog.sample(n=1, random_state=seed)
    lbl_path = os.path.join(label_dir, random_label.label.iloc[0])
    chip_path = os.path.join(chip_dir, random_label.chip.iloc[0])
    lbl_name = re.sub(".tif", "", random_label.label.iloc[0])
    m = leafmap.Map(
        zoom=17, center=random_label[["y", "x"]].iloc[0].to_list()
    )
    m.add_basemap("SATELLITE")
    m.add_raster(chip_path, bands=[1,2,3], layer_name='Image', 
                 zoom_to_layer=False)
    m.add_raster(lbl_path, layer_name=lbl_name,zoom_to_layer=False)
    return m

## Selecting a label catalog

The label catalog provided here provides all labels collected during the project, with the exception of sites where image quality was insufficient to label (note: additional labels may have been retained with low image quality).  Labels are provided in the following classes:

The labels have several distinct classes: 

-   Class 1a: A subset expert-labeled sites that were selected to serve as quality control sites (Q sites) in the labelling platform;

-   Class 1b: Expert-labeled sites not used in the labelling platform for Quality control;

-   Class 1c: Q control assignments completed by the labelling teams against Class 1a labels;

-   Class 1d: Sites corresponding to Class 1b sites that were digitized by 1-3 members of the labeling team;

-   Class 2: Ordinary mapping assignments, represented a single unique site mapped by a single labeller. An exception to this case of one labeller/one label were for those assignment marked as Untrusted, in which case it would have been mapped multiple times until the first approved assignment was completed. 

-   Class 4: Sites mapped by three separate labellers

The catalog provides a range of quality metrics associated with the labels, with the following exceptions; 

- For Class 1a and 1b, no Qscores are available
- For Class 1b, there are 153 sites for which no Rscore is available

All labels are preserved in the catalog, as they may be useful for different purposes, such as: 

- Contructing quality-weighted consensus maps to:
    1. Assess label uncertainty;
    2. Extract pixel-based samples from area of higher certainty
- Testing whether additional noisy labels added to training samples improve model performance;
- Pass multiple labels for the same location into a model as a form of data-based regularization

We provided an example here of one approach for selecting what is likely to be the best label for each site, using the following rules:

- Drop:
    - All assignments marked as Untrusted or Rejected;
    - No Class 1c labels;
- Keep:
    - All Class 1a assignments;
    - All Class 2 assignments;
    - The assignment with the highest Rscore for each site occuring in one or both of the Class 1b and Class 1d samples;
    - The assignment with the highest Rscore for each Class 4 site

Alternative approaches may also be used for finding the best sample, such as using the highest Qscore per sample, or multiple individual Q metrics, e.g. the highest combination of Area and N metrics. It may also be preferrable to take the Class 1c assignment with the highest Qscore or highest Rscore for each Class 1a site, in place of the Class 1a labels. 

### Read in catalog

In [4]:
catalog = pd.read_csv(Path(proj_dir) /\
                      "data/interim/label_catalog_allclasses.csv", 
                      low_memory=False)

### Apply filter

To follow the selection example describe above, we first queried the catalog to drop Untrusted and Rejected assignments, then defined a grouping list with the key for "whole" Classes to be retained set at "1a" and "2", and two separate groups for "best" selection. The first is for Classes "1b" and "1d", which means that if there are multiple assignments for the same site in those classes, the one with the highest metric (here Rscore) was selected. The second is for Class 4, and the same selection process was followed within that Class, picking the single assignment with the highest Rscore.  

Since some sites were mapped in Classes 1a and 1b, a final filtering is applied in which the Class 1a assignments were retained in preference to assignments for the same sites in Class 1b. 

In [5]:
catalog = catalog.query("status not in ['Untrusted', 'Rejected']")

groups = [
    {"whole": ["1a", "2"]}, 
    {"best": ["1b", "1d"]}, 
    {"best": "4"}
]

keep = ["name", "Class", "assignment_id", "Labeller", "status", "Score", 
        "N", "Area", "Qscore", "Rscore", "x", "y", "farea", "nflds", 
       "chip"]

label_catalog = filter_catalog(catalog, groups, "Rscore", keep)

# there are some duplicates of Class 1a in Classes 1b/d, so we'll drop those
label_catalog.drop_duplicates("name", inplace=True)

Extracting all of Class 1a and 2
Extracting best of Class 1b and 1d
Extracting best of Class 4


The final class counts were as follows

In [6]:
label_catalog.value_counts("Class")

Class
2     30772
4       990
1d      905
1a      797
1b      282
Name: count, dtype: int64

## Making label chips

The next step was to convert the polygons into chips. This requires the geoparquet file containing the field polygons. We will use the filtered label catalog to get the image chip that corresponds to the labelled site, and the convert the polygons for each site to a 3-class label that has the same dimensions as the image chip. 

In [7]:
fields = gpd.read_parquet(Path(proj_dir) /\
                          "data/processed/mapped_fields_final.parquet")

In [9]:
warnings.filterwarnings(
    'ignore', 
    message="Geometry is in a geographic CRS. Results from " +\
    "'buffer' are likely incorrect."
)
log = open(log_path, "a+")
print(f"\nStarting at {dt.now()}\n", file=log, flush=True)
lbls = []
for i, row in label_catalog.iterrows():
    lbls.append(threeclass_label(row, label_dir=label_dir, fields=fields, 
                                 log=log, overwrite=False))

print(f"\nFinished at {dt.now()}", file=log, flush=True)
log.close()

### Save catalog

In [11]:
label_catalog_final = pd.concat(lbls).reset_index(drop=True)
label_catalog_final.to_csv(Path(final_dir) / "label-catalog-filtered.csv")

In [12]:
label_catalog_final.iloc[0:2]

,name,Class,assignment_id,Labeller,status,Score,N,Area,Qscore,Rscore,x,y,farea,nflds,chip,label
0,NG0239934,2,122,10,Approved,NaN,NaN,NaN,0.61966,0.811741,6.4365,12.7425,0.449518,26,NG0239934-2022-03-15.tif,NG0239934-2022-03-15-122.tif
1,NG0781094,2,175,10,Approved,NaN,NaN,NaN,0.61966,0.811741,4.7015,11.4925,0.466465,13,NG0781094-2017-02-15.tif,NG0781094-2017-02-15-175.tif


## Display labels

The following viewer can be used to display labels against the images

In [20]:
view_random_label(label_catalog_final, label_dir, chip_dir)

Map(center=[7.6525, 8.4565], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [16]:
fields.shape

(825395, 6)